Importando Bibliotecas

In [129]:
import pandas as pd

Lendo Base de Dados

In [130]:
controle_path = 'bases'
def ler_arquivo (nome_arquivo):
    try:
        return pd.read_excel(f'{controle_path}/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'

In [131]:
nome_arquivo = 'dados de entregas'
df = ler_arquivo(nome_arquivo)

In [132]:
df

,id_entrega,filial,cliente,status,descricao_entrega,tipo_servico,valor_frete,data_postagem,prazo_previsto,data_prevista,data_entrega,dias_atraso
0,3619,SC,Americanas,ENTREGUE,Extravio temporário,ECONÔMICO,25.16,2026-01-09 20:43:34.883,5,2026-01-14 20:43:34.883,2026-01-15 20:43:34.883,1
1,4850,RJ,Amazon,ENTREGUE,Entrega reagendada,NORMAL,38.17,2026-01-06 20:43:34.889,5,2026-01-11 20:43:34.889,2026-01-10 20:43:34.889,0
2,3232,MG,Magazine Luiza,EM TRÂNSITO,Cliente ausente no local,NORMAL,311.62,2026-02-03 20:43:34.881,1,2026-02-04 20:43:34.881,2026-02-05 20:43:34.881,1
3,3944,RS,Amazon,EM TRÂNSITO,Cliente ausente no local,NORMAL,65.46,2026-01-17 20:43:34.885,7,2026-01-24 20:43:34.885,2026-01-26 20:43:34.885,2
4,2633,SP,Amazon,EM TRÂNSITO,Problema operacional na rota,ECONÔMICO,104.62,2026-02-03 20:43:34.875,2,2026-02-05 20:43:34.875,2026-02-05 20:43:34.875,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5195,3997,SC,Shopee,ENTREGUE,Entrega normal,NORMAL,38.12,2026-01-23 20:43:34.885,7,2026-01-30 20:43:34.885,2026-01-31 20:43:34.885,1
5196,3250,PR,Mercado Livre,ENTREGUE,Problema operacional na rota,ECONÔMICO,69.88,2026-01-19 20:43:34.881,2,2026-01-21 20:43:34.881,2026-01-19 20:43:34.881,0
5197,2834,PR,Magazine Luiza,EM TRÂNSITO,Extravio temporário,ECONÔMICO,152.43,2026-01-15 20:43:34.877,7,2026-01-22 20:43:34.877,2026-01-24 20:43:34.877,2
5198,507,MG,Amazon,EM TRÂNSITO,Entrega EXPRESSA,ECONÔMICO,27.43,2026-02-03 20:43:34.864,2,2026-02-05 20:43:34.864,2026-02-07 20:43:34.864,2


SLA

In [133]:
# Total de entregas sem atraso
sla_ok = df.loc[(df['dias_atraso'] == 0) & (df['status']=='ENTREGUE')].shape[0]
display(sla_ok)

# Total de entregas
total_entregas = df.loc[(df['status'] == 'ENTREGUE')].shape[0]
# display(total_entregas)

# Percentual de SLA
sla_percentual = (sla_ok/total_entregas)*100

f'{round(sla_percentual,2)}% de entregas dentro do prazo'

932

'37.24% de entregas dentro do prazo'

KPI - Entregas atrasadas

In [134]:
# Entregas atrasadas

sla_atrasadas = df.loc[(df['dias_atraso']>0) 
    & (df['status'] == 'ENTREGUE')
].shape[0]

atraso_percentual = (sla_atrasadas/total_entregas)*100

f'{round(atraso_percentual,2)}% de entregas atrasadas'

'62.76% de entregas atrasadas'

SLA - Identificando onde as operações funcionam melhor

In [135]:
sla_final = (
    df.loc[(df['status']=='ENTREGUE')]
).groupby('filial')['dias_atraso'].mean()

round(sla_final,2)

filial
MG    1.84
PR    1.92
RJ    1.81
RS    1.97
SC    1.91
SP    1.80
Name: dias_atraso, dtype: float64

In [136]:
sla_final = (
    df.loc[(df['status']=='ENTREGUE')]
).groupby('filial')['dias_atraso'].apply(
    lambda grupo: (grupo == 0).mean()*100
).sort_values(ascending=False)

round(sla_final,2).reset_index(name="% Entregas no prazo")

,filial,% Entregas no prazo
0,RJ,42.08
1,PR,38.34
2,MG,38.25
3,SP,35.76
4,RS,34.93
5,SC,34.76


In [137]:
# Validando palavra chave sem trabalhar com dataframe (tabela)
desc = "python é a melhor linguagem de todas"

palavras = desc.split()

try:
    palavras.index('python')
except ValueError as e:
    print(e)

Padronização da descrição

In [138]:
df['descricao_entrega'] = (
    df['descricao_entrega']
    .str.lower()
    .str.strip()
)

Separar entregas atrasadas

In [139]:
df_atrasos = df.loc[df['dias_atraso']>0].copy()
df_atrasos.shape

(3733, 12)

Categorizando as causas dos atrasos

In [140]:
df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('chuva|temporal|tempestade', na=False),
    "causa_atraso"
] = 'clima'

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('operacional|rota', na=False),
    "causa_atraso"
] = 'Problema operacional'

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('extravio', na=False),
    "causa_atraso"
] = 'extravio'

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('ausente', na=False),
    "causa_atraso"
] = 'ausencia do cliente'

df_atrasos['causa_atraso'] = df_atrasos['causa_atraso'].fillna('outros')

df_atrasos['causa_atraso']

0                  extravio
2       ausencia do cliente
3       ausencia do cliente
5                     clima
6                    outros
               ...         
5194                  clima
5195                 outros
5197               extravio
5198                 outros
5199                 outros
Name: causa_atraso, Length: 3733, dtype: object

Principais causas de atrasos

In [141]:
round(df_atrasos['causa_atraso'].value_counts(normalize=True) *100, 2).reset_index(name='percentual (%)')

,causa_atraso,percentual (%)
0,outros,49.58
1,extravio,13.37
2,ausencia do cliente,12.67
3,clima,12.67
4,Problema operacional,11.71


Atrasos por filial

In [142]:
causas_por_filiais = df_atrasos.groupby(['filial','causa_atraso']).size()\
        .reset_index(name='quantidade') \
        .sort_values(['filial','quantidade'], ascending=[True, False])

causas_por_filiais

,filial,causa_atraso,quantidade
4,MG,outros,274
2,MG,clima,84
1,MG,ausencia do cliente,83
3,MG,extravio,76
0,MG,Problema operacional,61
9,PR,outros,326
7,PR,clima,86
5,PR,Problema operacional,79
8,PR,extravio,74
6,PR,ausencia do cliente,69


Principais causas de atraso

In [146]:
total_filial = causas_por_filiais.groupby(['filial'])['quantidade']\
        .transform('sum')

causas_por_filiais['percentual'] = (
    round((causas_por_filiais['quantidade']/total_filial)*100, 2)
)

causas_por_filiais

,filial,causa_atraso,quantidade,percentual
4,MG,outros,274,47.40
2,MG,clima,84,14.53
1,MG,ausencia do cliente,83,14.36
3,MG,extravio,76,13.15
0,MG,Problema operacional,61,10.55
9,PR,outros,326,51.42
7,PR,clima,86,13.56
5,PR,Problema operacional,79,12.46
8,PR,extravio,74,11.67
6,PR,ausencia do cliente,69,10.88
